# Dirichlet-Multinomial Model
## Dirichlet Multinomial Distribution
The Dirichlet Multinomial distribution is a compound probability distribution, where a probability vector $\mathbf{p}$ is drawn from a Dirichlet distribution with parameter vector $\boldsymbol\alpha$, and an observation drawn from a multinomial distribution with probability vector $\mathbf{p}$ and number of trials $n$. The Dirichlet parameter vector captures the prior belief about the situation and can be seen as a pseudocount: observations of each outcome that occur before the actual data is collected. 

### Likelihood
Conceptually, we are making $N$ independent draws from a categorical distribution with $K$ categories. Let us represent the independent draws as random categorical variables $z_n$ for $n=1,\ldots,K$. let us denote the number of times a particular category $k$ has been seen (for $k=1,\ldots,K$) among all the categorical variables as $n_k$, and $\sum_k n_k = N$. Then we have two separate views onto this problem:

1. A set of $N$ categorical variables $z_1,\ldots,z_n$.
2. A single vector valued variable $\mathbf{x} = (n_1,\ldots, n_K)$ distributed according to a multinomial distribution.

The former case is a set of random variables specifying each *individual* outcome, and the latter is a variable specifying the *number* of outcomes of each of the $K$ categories. This distinction is important as the two cases have correspondingly different probability distributions.

The parameter of the categorical distribution is $\boldsymbol\theta = (\theta_1,\theta_2,\ldots,\theta_K)$, where $\theta_k$ is the probability of drawing value $k$; $\boldsymbol\theta$ is likewise the parameter of the multinomial distribution $p(\mathbf{x}|\boldsymbol\theta)$. Note that $\sum_k\theta_k = 1$.

Therefore, for an iid dataset $\mathcal{D}$, the likelihood function has the form:

$$
p(\mathcal{D}|\theta) = \prod_{k=1}^K\theta_k^{N_k}
$$

where $N_k = \sum_{i=1}^N\mathbb{I}(y_i = k)$ is the number of times event $k$ occurred. The likelihood for the multinomial model has the same form, up to a constant factor.

### Prior
Since the parameter vector lives in $K$-dimensional probability simplex, we need a prior that has support over that simplex. Ideally it would also be conjugate. Fortunately the Dirichlet distribution satisfies both criteria. So we use the following prior:

$$
\mathrm{Dir}(\boldsymbol\theta|\boldsymbol\alpha) = \frac{1}{B(\boldsymbol\alpha)}\prod_{k=1}^K\theta_k^{\alpha_k-1}\mathbb{I}(\mathbf{x}\in S_K)
$$

### Posterior
Multiplying the likelihood by the prior, we find that the posterior is also Dirichlet:

\begin{aligned}
p(\boldsymbol\theta|\mathcal{D}) & \propto p(\mathcal{D}|\boldsymbol\theta)p(\boldsymbol\theta) \\
& \propto \prod_{k=1}^K\theta_k^{N_k}\theta_k^{\alpha_k-1} = \prod_{k=1}^K\theta_k^{\alpha_k+N_k-1}\\
& = \mathrm{Dir}(\boldsymbol\theta|\alpha_1+N_1,\ldots,\alpha_k+N_k)
\end{aligned}

### MAP estimate
We can derive the mode (maximum) of this posterior by calculus. However, we need to enforce the constraint that $\sum_k\theta_k = 1$. We can do this by using a **Lagrange multipler**. The constrained objective function, or Lagrangian, is given by the log likelihood plus log prior plus the constraint:

$$
\mathcal{l}(\boldsymbol\theta,\lambda) = \sum_k N_k\log\theta_k + \sum_k(\alpha_k-1)\log\theta_k + \lambda\left(1-\sum_k\theta_k\right)
$$

Taking derivatives with respect to $\lambda$ yields the original constraint:

$$
\frac{\partial\mathcal{l}}{\partial\lambda} = \left(1-\sum_k\theta_k\right) = 0
$$

Taking derivatives with respect to $\theta_k$ yields

\begin{aligned}
\frac{\partial\mathcal{l}}{\partial\theta_k} = \frac{N_k+\alpha_k-1}{\theta_k}-\lambda & = 0\\
N_k + \alpha_k -1 & = \lambda\theta_k
\end{aligned}

We can solve for $\lambda$ using the sum-to-one constraint:

\begin{aligned}
\sum_k(N_k+\alpha_k-1) & = \lambda\sum_k\theta_k\\
N + \alpha_0 - K & = \lambda
\end{aligned}

where $\alpha_0\triangleq$$\sum_k\alpha_k$ is the equivalent sample size of the prior. Thus the MAP estimate is given by

$$
\hat{\theta}_k = \frac{N_k+\alpha_k-1}{N+\alpha_0-K}
$$

If we use a uniform prior, $\alpha_k = 1$, we recover the MLE:

$$
\hat{\theta}_k = N_k / N
$$

This is just the empirical fraction of time face $k$ shows up.

### Posterior predictive
The posterior predictive distribution for a single multinoulli trial is given by the following expression:

\begin{aligned}
p(X=j|\mathcal{D}) & = \int p(X=j|\boldsymbol\theta)p(\boldsymbol\theta|\mathcal{D})d\boldsymbol\theta \\
\end{aligned}

Before we can integrate this, let us introduce a small trick: For any $\boldsymbol\theta = (\theta_1,\ldots,\theta_m)$, let us define $\boldsymbol\theta_{\neg j} = (\theta_1,\ldots,\theta_{j-1},\theta_{j+1},\ldots,\theta_m)$, that is all values except $\theta_j$.

Now we always have the case that $\int p(\theta_\neg,\theta)d\theta_\neg = p(\theta)\int p(\theta_\neg)d\theta_\neg = p(\theta)$. (Marginalization rule for joint probabilities). So the above equation can be written explicitly as:

$$
p(X =j|\mathcal{D}) = \int_\theta p(X=j|\boldsymbol\theta)\left[\int_{\theta_\neg} p(\boldsymbol\theta_\neg,\theta_j|\mathcal{D})d\boldsymbol\theta_\neg\right]d\boldsymbol\theta
$$

Note in the single-trial multinomial case, the who term doesn't depend on $\boldsymbol\theta_\neg$, so we have

\begin{aligned}
p(X=j|\mathcal{D}) & = \int p(X=j|\theta_j)p(\theta_j)d\theta_j\\
& = \mathbb{E}(\theta_j|\mathcal{D}) = \frac{\alpha_j+N_j}{\sum_k(\alpha_k+N_k)} = \frac{\alpha_j+N_j}{\alpha_0 + N}
\end{aligned}

**NOTES**: The marginal distribution of the Dirichlet distribution $\mathrm{Dir}(\boldsymbol\theta|\boldsymbol\alpha)$ are beta distributions:

$$
\theta_j \sim \mathrm{Beta}(\alpha_j, \alpha_0-\alpha_j)
$$